In [1]:
import numpy as np
import os

In [2]:
def load_var(path, ip_len, op_len):
    ip_data_ls = []
    op_data_ls = []
    ip_data_ls1 = []
    sst = np.load(path + "/sst.npy")
    ssh = np.load(path + "/ssh.npy")
    nino34 = np.load(path + "/nino34.npy")
    print("data_shape:",sst.shape,ssh.shape,nino34.shape)
    for i in range(ip_len):
        idr = -ip_len + i + 1 - op_len if -ip_len + i + 1 - op_len != 0 else None
        ip_data_sst = sst[i:idr][:, :, :, np.newaxis]
        ip_data_ssh = ssh[i:idr][:, :, :, np.newaxis]
        ip_data_ls.append(ip_data_sst)
        ip_data_ls1.append(ip_data_ssh)
    for j in range(op_len):
        idl = j + ip_len
        idr = -op_len + j + 1 if -op_len + j + 1 != 0 else None
        op_data = nino34[idl:idr][:, np.newaxis]
        op_data_ls.append(op_data)
    ip_data_ls = np.concatenate(ip_data_ls, axis=3)
    ip_data_ls1 = np.concatenate(ip_data_ls1, axis=3)
    op_data_ls = np.concatenate(op_data_ls, axis=1)
    return ip_data_ls, ip_data_ls1, op_data_ls


ip_data_ls,ip_data_ls1, op_data_ls = load_var("../data/var_data", 3, 20)
ip_data_ls.shape, op_data_ls.shape,ip_data_ls1.shape


data_shape: (479, 24, 72) (479, 24, 72) (479,)


((457, 24, 72, 3), (457, 20), (457, 24, 72, 3))

In [3]:
import numpy as np
import pandas as pd
month_idx =  pd.date_range("1900-02-01","1930-12-30",freq="MS")
month_idx
ip_len=3
op_len=13
nino_month=1

ip_st_month = (nino_month - op_len - ip_len)%12 + 1
# 9 10 11
ip_st_month

10

In [4]:
ip_st_idx = ip_st_month - 2
ip_st_idx

8

In [5]:
month_idx[ip_st_idx:ip_st_idx+ip_len]

DatetimeIndex(['1900-10-01', '1900-11-01', '1900-12-01'], dtype='datetime64[ns]', freq='MS')

In [6]:
month_idx[ip_st_idx+ip_len+op_len-1]

Timestamp('1902-01-01 00:00:00', freq='MS')

In [7]:
# for idx in range(month_idx.shape[0]):
idx_begin = 9 -2
idx_nino1 = idx_begin + op_len + 3
month_idx[idx_begin],month_idx[idx_nino1]
idx_begin2 = idx_begin + 12
idx_nino2 = idx_nino1 + 12


In [8]:
month_idx[idx_begin2],month_idx[idx_nino2]

(Timestamp('1901-09-01 00:00:00', freq='MS'),
 Timestamp('1903-01-01 00:00:00', freq='MS'))

In [39]:
# month_idx =  pd.date_range("1850-01-01","2014-11-30",freq="MS")
month_idx =  pd.date_range("1850-01-01","1889-11-30",freq="MS")
def load_data(ip_len=3, op_len=13, nino_month=1, label="noVar"):
    if label == "noVar":
        bg_month = 2
    else:
        bg_month = 1
    ip_st_month = (nino_month - op_len - ip_len) % 12 + 1
    idx_begin = ip_st_month - bg_month
    nino_idx_begin = idx_begin + ip_len + op_len -1
    N = month_idx.shape[0]
    gap_year = np.abs((nino_month - op_len - ip_len) // 12)
    ip_data_ls = []
    op_data_ls = month_idx[nino_idx_begin::12]
    # print(gap_year)
    for eh in range(ip_len):
        ip_data = month_idx[idx_begin+eh::12]
        diff_len = ip_data.shape[0] - op_data_ls.shape[0]
        print(diff_len)
        if diff_len != 0:
            ip_data = ip_data[:-diff_len]
        print(ip_data.shape)
        ip_data_ls.append(ip_data[...,np.newaxis])
    ip_data_ls = np.concatenate(ip_data_ls,axis=-1)
    print(op_data_ls)
    return ip_data_ls,op_data_ls
ip_data_ls,op_data_ls = load_data()
print(ip_data_ls[-1],op_data_ls[-1])
print(ip_data_ls.shape,op_data_ls.shape)
# print(month_idx)

2
(38,)
2
(38,)
2
(38,)
DatetimeIndex(['1851-12-01', '1852-12-01', '1853-12-01', '1854-12-01',
               '1855-12-01', '1856-12-01', '1857-12-01', '1858-12-01',
               '1859-12-01', '1860-12-01', '1861-12-01', '1862-12-01',
               '1863-12-01', '1864-12-01', '1865-12-01', '1866-12-01',
               '1867-12-01', '1868-12-01', '1869-12-01', '1870-12-01',
               '1871-12-01', '1872-12-01', '1873-12-01', '1874-12-01',
               '1875-12-01', '1876-12-01', '1877-12-01', '1878-12-01',
               '1879-12-01', '1880-12-01', '1881-12-01', '1882-12-01',
               '1883-12-01', '1884-12-01', '1885-12-01', '1886-12-01',
               '1887-12-01', '1888-12-01'],
              dtype='datetime64[ns]', freq='12MS')
['1887-09-01T00:00:00.000000000' '1887-10-01T00:00:00.000000000'
 '1887-11-01T00:00:00.000000000'] 1888-12-01 00:00:00
(38, 3) (38,)


/tmp/ipykernel_5013/3522120182.py:23: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  ip_data_ls.append(ip_data[...,np.newaxis])


In [37]:
month_idx =  pd.date_range("1850-01-01","1889-11-30",freq="MS")
month_idx

DatetimeIndex(['1850-01-01', '1850-02-01', '1850-03-01', '1850-04-01',
               '1850-05-01', '1850-06-01', '1850-07-01', '1850-08-01',
               '1850-09-01', '1850-10-01',
               ...
               '1889-02-01', '1889-03-01', '1889-04-01', '1889-05-01',
               '1889-06-01', '1889-07-01', '1889-08-01', '1889-09-01',
               '1889-10-01', '1889-11-01'],
              dtype='datetime64[ns]', length=479, freq='MS')